In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from skimage.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from skimage.metrics import structural_similarity as ssim


# Define the paths to the original US images, tumor masks, and fused images
us_path = '/content/drive/MyDrive/Datasets/Segmentation/Dataset-B/Images'
mask_path = '/content/drive/MyDrive/Datasets/Segmentation/Dataset-B/masks'
fused_path = '/content/drive/MyDrive/Datasets/Segmentation/Dataset-B/Fused'

# Define the desired size of the images
image_size = (256, 256)

# Get a list of the image filenames in the fused directory
image_filenames = os.listdir(fused_path)

# Initialize lists to store the MSE and SSIM values for each image
us_fused_mse_list = []
us_fused_ssim_list = []
mask_fused_mse_list = []
mask_fused_ssim_list = []

# Metrics

In [ ]:
def calculate_psnr(original_image, fused_image):
    # Load the images
    img1 = cv2.imread(original_image)
    img2 = cv2.imread(fused_image)

    # Convert the images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Calculate the MSE
    mse = np.mean((img1_gray - img2_gray) ** 2)

    # Calculate the maximum pixel value
    max_pixel = 255.0

    # Calculate the PSNR
    if mse == 0:
        return "Same Image"
    else:
        psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        return psnr


# Wavelet based fusion

In [ ]:
def calculate_metrics(us_path, mask_path, fused_path):
    # Initialize arrays to store MSE and SSIM values
    mse_values = []
    psnr_values = []
    ssim_values = []

    # Loop over each image file in the original US images and mask directories
    for filename in os.listdir(us_path):
        # Read in the original US image and mask, and the corresponding fused image
        us_image = cv2.imread(os.path.join(us_path, filename))
        mask_image = cv2.imread(os.path.join(mask_path, filename))
        fused_image = cv2.imread(os.path.join(fused_path, filename))

        # Normalize the image data to [0, 1] range
        us_image_norm = cv2.normalize(us_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        mask_image_norm = cv2.normalize(mask_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        fused_image_norm = cv2.normalize(fused_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Calculate MSE and PSNR between the original US image and the fused image
        mse = np.mean((us_image_norm - fused_image_norm)**2)
        psnr = calculate_psnr(os.path.join(us_path, filename), os.path.join(fused_path, filename))

        mse_values.append(mse)
        psnr_values.append(psnr)

        # Calculate SSIM between the original US image and mask, and the fused image
        ssim_value = ssim(us_image_norm, fused_image_norm, channel_axis=2)
        ssim_values.append(ssim_value)

    # Compute the mean MSE, PSNR, and SSIM values over all images
    mean_mse = np.mean(mse_values)
    mean_psnr = np.mean(psnr_values)
    mean_ssim = np.mean(ssim_values)

    return mean_mse, mean_psnr, mean_ssim


In [ ]:
# Call the function to calculate the metrics
mean_mse, mean_psnr, mean_ssim = calculate_metrics(us_path, mask_path, fused_path)

# Print the results
print("Mean MSE:", mean_mse)
print("Mean PSNR:", mean_psnr)
print("Mean SSIM:", mean_ssim)

Mean MSE: 0.036059055
Mean PSNR: 27.99243999516554
Mean SSIM: 0.7666144


# PCA Fusion

In [ ]:
# Define a function to perform PCA-based image fusion
def pca_fusion(us_image, mask_image):
    # Convert the images to grayscale
    us_gray = cv2.cvtColor(us_image, cv2.COLOR_BGR2GRAY)
    mask_gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)

    # Compute the covariance matrix of the two images
    cov = np.cov(np.stack([us_gray.flatten(), mask_gray.flatten()], axis=0))

    # Compute the eigenvectors and eigenvalues of the covariance matrix
    eig_vals, eig_vecs = np.linalg.eig(cov)

    # Sort the eigenvectors in descending order of eigenvalues
    sort_indices = np.argsort(eig_vals)[::-1]
    eig_vecs_sorted = eig_vecs[:, sort_indices]

    # Compute the fused image as a linear combination of the two input images
    us_weight = eig_vecs_sorted[0, 0] / (eig_vecs_sorted[0, 0] + eig_vecs_sorted[0, 1])
    mask_weight = eig_vecs_sorted[0, 1] / (eig_vecs_sorted[0, 0] + eig_vecs_sorted[0, 1])
    fused_image = us_weight * us_gray + mask_weight * mask_gray

    # Normalize the fused image to [0, 255] range
    fused_image = cv2.normalize(fused_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return fused_image

In [ ]:
# Define a function to calculate MSE, SSIM and PSNR
def calculate_metrics(us_path, mask_path):
    # Initialize arrays to store MSE, PSNR and SSIM values
    mse_values = []
    psnr_values = []
    ssim_values = []

    # Loop over each image file in the original US images and mask directories
    for filename in os.listdir(us_path):
        # Read in the original US image and mask, and perform PCA-based fusion
        us_image = cv2.imread(os.path.join(us_path, filename))
        mask_image = cv2.imread(os.path.join(mask_path, filename))
        fused_image = pca_fusion(us_image, mask_image)

        # Convert the original US image to grayscale
        us_image_gray = cv2.cvtColor(us_image, cv2.COLOR_BGR2GRAY)

        # Normalize the image data to [0, 1] range
        us_image_norm = cv2.normalize(us_image_gray, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        mask_image_norm = cv2.normalize(mask_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        fused_image_norm = cv2.normalize(fused_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Calculate MSE and PSNR between the original US image and mask, and the fused image
        mse = np.mean((us_image_norm - fused_image_norm)**2)
        psnr = calculate_psnr(os.path.join(us_path, filename), os.path.join(fused_path, filename))

        mse_values.append(mse)
        psnr_values.append(psnr)

        # Calculate SSIM between the original US image and mask, and the fused image
        ssim_value = ssim(us_image_norm, fused_image_norm)
        ssim_values.append(ssim_value)

    # Compute the mean MSE, PSNR, and SSIM values over all images
    mean_mse = np.mean(mse_values)
    mean_psnr = np.mean(psnr_values)
    mean_ssim = np.mean(ssim_values)

    return mean_mse, mean_psnr, mean_ssim


In [ ]:
# Call the function to calculate the metrics
mean_mse, mean_psnr, mean_ssim = calculate_metrics(us_path, mask_path)

# Print the results
print("Mean MSE:", mean_mse)
print("Mean PSNR:", mean_psnr)
print("Mean SSIM:", mean_ssim)

Mean MSE: 0.073905684
Mean PSNR: 27.99243999516554
Mean SSIM: 0.5445152100384141


# DCT Fusion

In [ ]:
def dct_fusion(us_image, mask_image):
    # Convert the images to grayscale
    us_gray = cv2.cvtColor(us_image, cv2.COLOR_BGR2GRAY)
    mask_gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)

    # Compute the Discrete Cosine Transform (DCT) of the two images
    us_dct = cv2.dct(np.float32(us_gray)/255.0)
    mask_dct = cv2.dct(np.float32(mask_gray)/255.0)

    # Compute the magnitude of the DCT coefficients
    us_mag = np.abs(us_dct)
    mask_mag = np.abs(mask_dct)

    # Compute the fused image by taking the maximum magnitude of each DCT coefficient
    fused_mag = np.maximum(us_mag, mask_mag)

    # Compute the Inverse Discrete Cosine Transform (IDCT) of the fused image
    fused_image = cv2.idct(fused_mag)*255.0

    # Normalize the fused image to [0, 255] range
    fused_image = cv2.normalize(fused_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return fused_image

In [ ]:
# Define a function to calculate MSE, SSIM and PSNR
def calculate_metrics(us_path, mask_path):
    # Initialize arrays to store MSE, PSNR and SSIM values
    mse_values = []
    psnr_values = []
    ssim_values = []

    # Loop over each image file in the original US images and mask directories
    for filename in os.listdir(us_path):
        # Read in the original US image and mask, and perform PCA-based fusion
        us_image = cv2.imread(os.path.join(us_path, filename))
        mask_image = cv2.imread(os.path.join(mask_path, filename))
        fused_image = dct_fusion(us_image, mask_image)

        # Convert the original US image to grayscale
        us_image_gray = cv2.cvtColor(us_image, cv2.COLOR_BGR2GRAY)

        # Normalize the image data to [0, 1] range
        us_image_norm = cv2.normalize(us_image_gray, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        mask_image_norm = cv2.normalize(mask_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        fused_image_norm = cv2.normalize(fused_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Calculate MSE and PSNR between the original US image and mask, and the fused image
        mse = np.mean((us_image_norm - fused_image_norm)**2)
        psnr = calculate_psnr(os.path.join(us_path, filename), os.path.join(fused_path, filename))

        mse_values.append(mse)
        psnr_values.append(psnr)

        # Calculate SSIM between the original US image and mask, and the fused image
        ssim_value = ssim(us_image_norm, fused_image_norm)
        ssim_values.append(ssim_value)

    # Compute the mean MSE, PSNR, and SSIM values over all images
    mean_mse = np.mean(mse_values)
    mean_psnr = np.mean(psnr_values)
    mean_ssim = np.mean(ssim_values)

    return mean_mse, mean_psnr, mean_ssim


In [ ]:
# Call the function to calculate the metrics
mean_mse, mean_psnr, mean_ssim = calculate_metrics(us_path, mask_path)

# Print the results
print("Mean MSE:", mean_mse)
print("Mean PSNR:", mean_psnr)
print("Mean SSIM:", mean_ssim)

Mean MSE: 0.092999764
Mean PSNR: 27.99243999516554
Mean SSIM: 0.16395039541779605


# Gradient Fusion

In [ ]:
def gradient_fusion(us_image, mask_image):
    # Convert the images to grayscale
    us_gray = cv2.cvtColor(us_image, cv2.COLOR_BGR2GRAY)
    mask_gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)

    # Compute the gradient magnitude of each image
    us_grad_x = cv2.Sobel(us_gray, cv2.CV_64F, 1, 0, ksize=3)
    us_grad_y = cv2.Sobel(us_gray, cv2.CV_64F, 0, 1, ksize=3)
    us_grad_mag = np.sqrt(us_grad_x**2 + us_grad_y**2)

    mask_grad_x = cv2.Sobel(mask_gray, cv2.CV_64F, 1, 0, ksize=3)
    mask_grad_y = cv2.Sobel(mask_gray, cv2.CV_64F, 0, 1, ksize=3)
    mask_grad_mag = np.sqrt(mask_grad_x**2 + mask_grad_y**2)

    # Compute the fused image as a weighted sum of the gradient magnitudes
    fused_grad_mag = 0.5*us_grad_mag + 0.5*mask_grad_mag

    # Normalize the fused image to [0, 255] range
    fused_image = cv2.normalize(fused_grad_mag, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return fused_image

In [ ]:
# Define a function to calculate MSE, SSIM and PSNR
def calculate_metrics(us_path, mask_path):
    # Initialize arrays to store MSE, PSNR and SSIM values
    mse_values = []
    psnr_values = []
    ssim_values = []

    # Loop over each image file in the original US images and mask directories
    for filename in os.listdir(us_path):
        # Read in the original US image and mask, and perform PCA-based fusion
        us_image = cv2.imread(os.path.join(us_path, filename))
        mask_image = cv2.imread(os.path.join(mask_path, filename))
        fused_image = gradient_fusion(us_image, mask_image)

        # Convert the original US image to grayscale
        us_image_gray = cv2.cvtColor(us_image, cv2.COLOR_BGR2GRAY)

        # Normalize the image data to [0, 1] range
        us_image_norm = cv2.normalize(us_image_gray, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        mask_image_norm = cv2.normalize(mask_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        fused_image_norm = cv2.normalize(fused_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Calculate MSE and PSNR between the original US image and mask, and the fused image
        mse = np.mean((us_image_norm - fused_image_norm)**2)
        psnr = calculate_psnr(os.path.join(us_path, filename), os.path.join(fused_path, filename))

        mse_values.append(mse)
        psnr_values.append(psnr)

        # Calculate SSIM between the original US image and mask, and the fused image
        ssim_value = ssim(us_image_norm, fused_image_norm)
        ssim_values.append(ssim_value)

    # Compute the mean MSE, PSNR, and SSIM values over all images
    mean_mse = np.mean(mse_values)
    mean_psnr = np.mean(psnr_values)
    mean_ssim = np.mean(ssim_values)

    return mean_mse, mean_psnr, mean_ssim


In [ ]:
# Call the function to calculate the metrics
mean_mse, mean_psnr, mean_ssim = calculate_metrics(us_path, mask_path)

# Print the results
print("Mean MSE:", mean_mse)
print("Mean PSNR:", mean_psnr)
print("Mean SSIM:", mean_ssim)

Mean MSE: 0.076086484
Mean PSNR: 27.99243999516554
Mean SSIM: 0.24647591843617253
